In [ ]:
import torch.nn as nn
import torch 
from torch.nn import LSTM,RNN,GRU,BatchNorm1d,Linear,Flatten,Dropout,Conv1d
import torch.nn.functional as F
import torch.optim as optim
import keras
from keras.utils import to_categorical
#定义模型
class logdifmodel_basebert(nn.Module):
    def __init__(self,input_shape:int) -> None:
        super().__init__()
        self.lstm1=LSTM(input_size=input_shape,hidden_size=64)
        self.lstm2=LSTM(input_size=64,hidden_size=32)
        self.flatten=Flatten()
        self.dropout=Dropout(0.1)
        self.dense1=Linear(3200,15936,bias=True)
    
    #前向传播
    def forward(self,x):
        out,(hidden,c)=self.lstm1(x)
        out,(hidden,c)=self.lstm2(out)
        flatten_out=self.flatten(out)
        drop_out=self.dropout(flatten_out)
        output=F.softmax(self.dense1(drop_out))
        return output
 
def get_bestmodel(input_size:int,train_x:torch.tensor,train_y:torch.tensor,learning_rate=0.1,beta=(0.9,0.98),epochs=100,batch_size=128):

    #train_x,train_y为torch.tensor类型，
    logdif_model=logdifmodel_basebert(input_size)
    #定义损失函数
    criterion=nn.CrossEntropyLoss()
    #定义优化器
    optimizer=optim.Adam(logdif_model.parameters(),lr=learning_rate,betas=beta)
    #确定多少个batch
    n_batchs=len(train_x)//batch_size
    #开始进行训练
    for epoch in range(epochs):
        
        for n_batch in range(n_batchs):
            batch_input=train_x[n_batchs*batch_size:n_batchs*batch_size+batch_size]
            batch_labels=train_y[n_batchs*batch_size:n_batchs*batch_size+batch_size]
            batch_output=logdif_model(batch_input)
            #计算损失
            loss=criterion(batch_output,batch_labels)
            #将梯度重置为0
            optimizer.zero_grad()
            #反向梯度传播
            loss.backward()
            #优化参数
            optimizer.step()
            #计算正确率
            y_pre=torch.max(logdif_model(train_features),1)[1]
            acc=float((y_pre==labels).sum())/float(labels.shape[0])
            print(str(loss),'acc of this model is :%.4f'%acc)
    return logdif_model

In [ ]:
from keras import models
from keras import Sequential,layers
from keras.layers import Dense,LSTM,SimpleRNN,Flatten


model=models.Sequential()
model.add(LSTM(64,return_sequences=True,input_shape=(100,768)))
#model.add(LSTM(32,activation='tanh'))
model.add(Flatten())
model.add(Dense(15936,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics='acc')
model.fit(keras_data,keras_labels,epochs=2,batch_size=20,validation_data=0.2)